In [1]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

# FLOWPACK


A. Class Flow:

  1. Componenets:

  i. *init* :
    
    - Requirements:

      - Density
      
      - Velocity
      
      - Pressure

      - Elevation

      - Viscosity

        - *Don't leave empty, set equal to zero for Nonviscous Models*

      - Array
        
        - Pass all these arguments in order, of course, into an array

    All these variables default into their respecitve sympy symbol as in common usage by default.

    Only Consistent Dimensionless *units* are supported.
    
  ii. *show*:

    *No arguments required*

    Prints out various flow parameters. Note that pipe parameters are not included.

  iii. *unravel*:

    *No arguments required*

    Returns an array in prcisely the order of parameters for *flow.init*

    Convert to list if you need the result for appending and extending

  iv. *eval_Re*:
  
    - Requirements:
      - pipe_diameter: D

    - Function:

      Returns the Reynolds Number of flow at a specific pipe diameter (Velocity is specified by the init method)

  v. *eval_visocus_head*:

    Returns the Visocus head for the flow (Major losses)
    
    - Requirements:

      - pipe_diameter : D
      
      - pipe_roughness : ϵ

      - pipe_length : L

    - Returns:

      - Sympy.symbol Object : If all variables are known

      - Sympy.real or float Object : If something is unknown

  vi. *solve*:

    *Solves the flow with the total head (mechanical + elvation) set equal to 0*

    - Requirements:

      - pipe_diameter : D
      
      - pipe_roughness : ϵ

      - pipe_length : L

    - Returns:

      - Sympy.symbol Object : If all variables are known

      - Sympy.Real or float Object : If something is unknown

In [36]:
''' 

NOTE:

1. When Specifying values always supply them as floating point values
2. When Specifying Nonvisocus flow, set flow.viscosity = 0, Else it won't work
'''

g = 9.81

class flow:
  # If you specify the array variable then DONOT specify any other variable
  def __init__(self, density=sym.Symbol(r'rho'), velocity=sym.Symbol(r'V'), elevation=sym.Symbol(r'h'), pressure=sym.Symbol(r'P'), viscosity=sym.Symbol(r'\mu'), array=np.array([])):
    if not(np.array(array).size == 0):
      density = array[0]
      velocity = array[1]
      elevation = array[2]
      pressure = array[3]
      viscosity = array[4]

    self.density = density
    self.velocity = velocity
    self.elevation = elevation
    self.pressure = pressure
    self.viscosity = viscosity

  # prints the flow
  def show(self):
    print(f'\nDensity: {self.density}\nPrssure: {self.pressure}\nVelocity: {self.velocity}\nElevation: {self.elevation}\nViscosity: {self.viscosity}')
    return self

  def unravel(self):
    return np.array([self.density, self.velocity, self.elevation, self.pressure, self.viscosity])

  # Evaluate the Reynolds Number
  def eval_Re(self, pipe_diameter): # Pipe diameter can change continuously throughout a flow so here is where I fancy putting it
    return self.density * self.velocity * pipe_diameter / self.viscosity

  # Evaluate the viscous head
  def eval_viscous_head(self, pipe_diameter=sym.Symbol(r'D'), pipe_roughness=sym.Symbol(r'\epsilon'), pipe_length=sym.Symbol(r'L'), mode='unknown'):

    if mode=='laminar':
      return self.eval_Re(pipe_diameter)
    
    Re = self.eval_Re(pipe_diameter)

    if type(Re) == int and Re < 2300:
      return 64/self.eval_Re(pipe_diameter) * pipe_length / pipe_diameter * self.velocity **2 / (2 * g)

    # Default to turbulent flow - flow is propably turbulent anyways, and this always helps out as a safety consideration
    # I use the simpler approximation so it is more *solveable*
    else:
      return 0.316/(Re ** 0.25) * pipe_length / pipe_diameter * self.velocity **2/ 2 * g
  
  # Calclates flow parameters with the total head = 0
  def solve(self, pipe_diameter=sym.Symbol(r'D'), pipe_roughness=sym.Symbol(r'\epsilon'), pipe_length=sym.Symbol(r'L')): # Solve using bernoulli & darcy-weisbach method
    # Need these values because I am going to give these values later on to sym.solve()
    unsolved_var = self.unravel()[(np.vectorize(type)(self.unravel()) == type(sym.Symbol(r'x')))] # Fetch all the unsolved variables

    # This is needed for the generating the final result array
    solved_var = self.unravel()[(np.vectorize(type)(self.unravel()) ==  float)] # Fetch all 'provided' floating point varialbes

    # Finds all the indices of all the solved and unsolved componenets
    solved_index, unsolved_index = [], []
    for i, j in enumerate(self.unravel()):
        if j in unsolved_var:
          unsolved_index.append(i)
        else:
          solved_index.append(i)

    if self.viscosity == 0: # Non Viscous Flow
      energy_eqn = self.velocity ** 2 / (2 * g) + self.elevation + self.pressure / self.density
    
    
    else: # Viscous Flow
      energy_eqn = self.velocity ** 2 / (2 * g) + self.elevation + self.pressure / self.density - self.eval_viscous_head(pipe_diameter, pipe_roughness, pipe_length)
      unsolved_var, pipe_vars = list(unsolved_var), np.array([pipe_diameter, pipe_roughness, pipe_length])
      new_unsolved_vars = pipe_vars[np.vectorize(type)(pipe_vars) == sym.Symbol]
                    # Fetch new unsolved variables

      # Unsolved Variables and their indices
      new_unsolved_vars_index = []
      for i, j in enumerate(pipe_vars):
        if j in new_unsolved_vars:
          new_unsolved_vars_index.append(5 + i)

      # Give a summary of viscous properties
      print("Summary of Data Fed into the Solver: ")
      print(f"Pipe Diameter: {pipe_diameter}, Pipe Roughness; {pipe_roughness}, Pipe Length: {pipe_length}")


      unsolved_var.extend(list(new_unsolved_vars)) # Unsolved variables - extended
      unsolved_index.extend(new_unsolved_vars_index) # Unsolved variables index - extended


    # Unsolved Varibles Solved "for"
    print(f"\n\nSolving...\n{energy_eqn} = 0\n")
    print(f"Unsolved Variables: ", unsolved_var)
    print(f"Provided Variales: ", solved_var)
    unsolved_var_solved = sym.solve(energy_eqn, [i for i in unsolved_var])  # Why does this stuck here ????
    
    # Array for generating the result
    ravel_array = [0, 0, 0, 0, 0, 0, 0, 0] # Leaving this empty is worse

    # What does this part do?
    if np.array(unsolved_index).size > 1:
      for i, j in enumerate(unsolved_index):
        ravel_array[j] = unsolved_var_solved[0][i]

    if np.array(unsolved_index).size >= 1:
      for i, j in enumerate(solved_index):
        ravel_array[j] = solved_var[i]

    # Give a warning if any parameter turns out to be , umm. a Complex Number
    # I really ain't the BFF of iota (sorry Eurler)
    for i, j  in enumerate(ravel_array):
      if type(j) == sym.Mul:
          vars = list(flow().unravel()) # Fix this so that var becomes the symbol for the particular variable that became Mul
          vars.extend(sym.symbols(r'D \epsilon L'))
          
          var = vars[i]
          print("Invalid Type Warning at index (or Symbol): ", var, " is complex. with value ", j)

    return flow(array=ravel_array)

## Examples:

In [38]:
x = flow(density=1000., viscosity=0.1, pressure=0., elevation=2.)
y = x.solve(pipe_diameter=.1, pipe_roughness=1., pipe_length=1.)

# There is some bug in the configuration for finding out the velocity
# Pipe Length & Velocity -> Not given
# Figures out Pipe Length = f(V)

y.show()
print('\n')

Summary of Data Fed into the Solver: 
Pipe Diameter: 0.1, Pipe Roughness; 1.0, Pipe Length: 1.0


Solving...
0.0509683995922528*V**2 - 2.75629751997213*V**1.75 + 2.0 = 0

Unsolved Variables:  [V]
Provided Variales:  [1000.0 2.0 0.0 0.1]

Density: 1000.0
Prssure: 0.0
Velocity: 0
Elevation: 2.0
Viscosity: 0.1




In [39]:
V = sym.Symbol(r'V')
eqn = 0.0509683995922528*V**2 - 2.75629751997213*V**1.75 + 2.0

sym.solve(eqn, [V])

[0.841078084963806, 8552643.89014053]